# 🧠 Mistral 7B Task Breakdown System for ShowerLog

**Superior AI-powered task decomposition using Mistral-7B-Instruct-v0.2**

Environment Setup and Token Configuration

In [ ]:
import subprocess
import sys
import torch
import os

print("Installing dependencies...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", 
               "torch", "transformers", "accelerate", "bitsandbytes", 
               "flask", "flask-cors", "pyngrok", "sentencepiece", "psutil", "requests"], check=True)

print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# 🔑 CRITICAL: Add your authentication tokens here!
print("\n🔑 Token Configuration Required:")
print("=" * 50)

# 1. Hugging Face Token (Required for Mistral model access)
# Get your token from: https://huggingface.co/settings/tokens
# You need 'Read' permission to access the Mistral model
HUGGINGFACE_TOKEN = "YOUR_HUGGINGFACE_TOKEN_HERE"

# 2. Ngrok Token (Required for public URL tunnel)
# Get your token from: https://dashboard.ngrok.com/get-started/your-authtoken
# Free accounts get 1 tunnel, which is perfect for this project
NGROK_TOKEN = "YOUR_NGROK_TOKEN_HERE"

# Token validation and setup
if HUGGINGFACE_TOKEN == "YOUR_HUGGINGFACE_TOKEN_HERE":
    print("❌ MISSING: Hugging Face Token")
    print("📝 Steps to get your token:")
    print("   1. Go to: https://huggingface.co/settings/tokens")
    print("   2. Click 'New token'")
    print("   3. Name it 'showerlog-mistral'")
    print("   4. Select 'Read' permission")
    print("   5. Copy the token and paste it above")
    print("   6. Replace 'YOUR_HUGGINGFACE_TOKEN_HERE' with your actual token")
    print("   Example: hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
    raise ValueError("❌ Hugging Face token required for Mistral model access!")

if NGROK_TOKEN == "YOUR_NGROK_TOKEN_HERE":
    print("❌ MISSING: Ngrok Token")
    print("📝 Steps to get your token:")
    print("   1. Go to: https://dashboard.ngrok.com/get-started/your-authtoken")
    print("   2. Sign up for free if you don't have an account")
    print("   3. Copy your authtoken from the dashboard")
    print("   4. Replace 'YOUR_NGROK_TOKEN_HERE' with your actual token")
    print("   Example: 2abcdefghijk1234567890mnopqrstuvwxyz_abcdefghijklmnopqrstuvwxyz123")
    raise ValueError("❌ Ngrok token required for public URL access!")

# Set environment variables for authentication
os.environ['HUGGINGFACE_HUB_TOKEN'] = HUGGINGFACE_TOKEN
os.environ['HF_TOKEN'] = HUGGINGFACE_TOKEN

# Set up ngrok authentication
from pyngrok import ngrok
ngrok.set_auth_token(NGROK_TOKEN)

print("✅ Environment setup complete!")
print("✅ Hugging Face token configured")
print("✅ Ngrok token configured")
print("🚀 Ready to load Mistral 7B model!")

Load Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import warnings
import psutil
warnings.filterwarnings('ignore')

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# Check available memory and hardware
print(f"Available RAM: {psutil.virtual_memory().available / (1024**3):.1f} GB")
print(f"CUDA Available: {torch.cuda.is_available()}")

# Load tokenizer first
print("🔄 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
print("✅ Tokenizer loaded successfully!")

# Smart model loading strategy for Kaggle environment
print("🔄 Loading model with Kaggle-optimized configuration...")

try:
    # Method 1: Try GPU loading without quantization (works better in Kaggle)
    if torch.cuda.is_available():
        print("🚀 Attempting GPU loading without quantization...")
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            trust_remote_code=True,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True
        )
        print("✅ Mistral 7B loaded successfully on GPU!")
    else:
        raise Exception("CUDA not available, falling back to CPU")
        
except Exception as e:
    print(f"⚠️ GPU loading failed: {str(e)}")
    print("🔄 Trying CPU loading with optimizations...")
    
    try:
        # Method 2: CPU loading with memory optimizations
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="cpu",
            trust_remote_code=True,
            torch_dtype=torch.float32,
            low_cpu_mem_usage=True
        )
        print("✅ Mistral 7B loaded successfully on CPU!")
        
    except Exception as e2:
        print(f"⚠️ CPU loading failed: {str(e2)}")
        print("🔄 Trying minimal loading configuration...")
        
        # Method 3: Last resort - minimal configuration
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            trust_remote_code=True,
            low_cpu_mem_usage=True
        )
        print("✅ Mistral 7B loaded with minimal configuration!")

print(f"📍 Model device: {next(model.parameters()).device}")
print(f"📊 Model memory footprint: ~{sum(p.numel() * p.element_size() for p in model.parameters()) / (1024**3):.1f} GB")
print("🎯 Ready for intelligent task breakdown generation!")

Advanced Infinite Task Breakdown AI

In [1]:
import json
import re
import time
from typing import Dict, Any, List

class MistralInfiniteTaskBreakdownAI:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        # Fast generation config optimized for Kaggle
        self.generation_config = {
            "max_new_tokens": 500,  # Reduced for faster generation
            "temperature": 0.3,     # Lower for more focused output
            "top_p": 0.85,         # Slightly more focused
            "do_sample": True,
            "pad_token_id": tokenizer.eos_token_id,
            "early_stopping": True, # Stop early when EOS is generated
            "num_beams": 1,        # No beam search for speed
        }
        self.complexity_weights = {
            'simple': 0.3,
            'moderate': 0.6,
            'complex': 0.8,
            'enterprise': 1.0
        }
        self.project_contexts = {
            # Technology & Development
            'software': 'software development, coding, technical implementation, programming, system architecture',
            'web': 'web development, frontend, backend, full-stack, responsive design, user experience',
            'mobile': 'mobile app development, iOS, Android, cross-platform, React Native, Flutter',
            'ai': 'artificial intelligence, machine learning, data science, neural networks, automation',
            'cybersecurity': 'information security, penetration testing, vulnerability assessment, compliance',
            'devops': 'deployment, CI/CD, infrastructure, cloud computing, containerization, monitoring',
            
            # Business & Finance
            'business': 'business strategy, market analysis, financial planning, operations, growth',
            'startup': 'startup development, MVP creation, fundraising, pitch decks, market validation',
            'finance': 'financial management, budgeting, investment, accounting, tax planning, wealth building',
            'marketing': 'digital marketing, SEO, social media, content marketing, brand building, campaigns',
            'sales': 'sales strategy, lead generation, customer acquisition, CRM, sales funnels',
            'consulting': 'business consulting, strategic advice, process optimization, change management',
            'ecommerce': 'online retail, marketplace development, inventory management, customer service',
            
            # Creative & Arts
            'creative': 'artistic creation, design, content development, visual arts, creative expression',
            'design': 'graphic design, UI/UX design, product design, branding, typography, visual identity',
            'photography': 'photography techniques, photo editing, portfolio building, commercial photography',
            'videography': 'video production, filming, editing, cinematography, storytelling, post-production',
            'music': 'music composition, recording, mixing, mastering, live performance, music business',
            'writing': 'creative writing, copywriting, blogging, journalism, storytelling, publishing',
            'art': 'fine arts, painting, drawing, sculpture, digital art, art history, gallery management',
            'fashion': 'fashion design, styling, trend analysis, fashion business, sustainable fashion',
            
            # Health & Wellness
            'fitness': 'physical fitness, workout planning, strength training, cardio, nutrition, wellness',
            'nutrition': 'meal planning, diet optimization, healthy eating, weight management, supplements',
            'mental_health': 'stress management, mindfulness, meditation, therapy, emotional wellness',
            'healthcare': 'medical practice, patient care, healthcare administration, medical research',
            'beauty': 'skincare, makeup, beauty treatments, cosmetics, personal grooming, beauty business',
            'wellness': 'holistic health, lifestyle optimization, work-life balance, self-care practices',
            
            # Education & Learning
            'learning': 'education, skill development, knowledge acquisition, study techniques, certification',
            'teaching': 'curriculum development, lesson planning, educational methods, student engagement',
            'research': 'academic research, data collection, analysis, thesis writing, scientific method',
            'language': 'language learning, linguistics, translation, communication skills, cultural studies',
            'academics': 'academic planning, degree completion, scholarship applications, academic writing',
            
            # Sports & Recreation
            'sports': 'athletic training, sports performance, coaching, team management, sports medicine',
            'outdoor': 'outdoor activities, hiking, camping, adventure sports, nature exploration',
            'gaming': 'game development, esports, gaming content creation, competitive gaming',
            'hobby': 'hobby development, crafts, collecting, recreational activities, personal interests',
            'travel': 'travel planning, itinerary creation, budget travel, cultural exploration, tourism',
            
            # Performing Arts & Entertainment
            'dancing': 'dance training, choreography, performance preparation, dance styles, dance business',
            'theater': 'acting, theater production, stage management, script writing, performance arts',
            'entertainment': 'event planning, entertainment industry, content creation, media production',
            'comedy': 'comedy writing, stand-up performance, humor development, entertainment content',
            
            # Personal Development & Relationships
            'relationships': 'relationship building, communication skills, dating, marriage, family dynamics',
            'personal': 'personal development, goal setting, habit formation, life coaching, self-improvement',
            'social': 'social skills, networking, community building, social media presence, influence',
            'parenting': 'child development, parenting strategies, education planning, family management',
            'dating': 'dating strategies, relationship building, social confidence, romantic connections',
            
            # Home & Lifestyle
            'cooking': 'culinary skills, recipe development, meal preparation, food safety, cooking techniques',
            'gardening': 'plant care, garden design, sustainable gardening, urban farming, landscaping',
            'diy': 'do-it-yourself projects, home improvement, crafts, repairs, building, woodworking',
            'home': 'home organization, interior design, cleaning, maintenance, home automation',
            'sustainability': 'eco-friendly living, renewable energy, waste reduction, sustainable practices',
            
            # Career & Professional
            'career': 'career development, job searching, professional growth, skill building, networking',
            'job_hunting': 'resume writing, interview preparation, job applications, career transitions',
            'freelancing': 'freelance business, client acquisition, project management, independent work',
            'leadership': 'leadership development, team management, organizational skills, executive coaching',
            'productivity': 'time management, efficiency optimization, workflow improvement, productivity systems',
            
            # Science & Technology
            'science': 'scientific research, experimentation, data analysis, laboratory work, innovation',
            'engineering': 'engineering design, problem solving, technical analysis, system optimization',
            'environment': 'environmental science, conservation, climate action, sustainability projects',
            'astronomy': 'space science, astronomy observation, astrophotography, space exploration',
            
            # Manufacturing & Trades
            'manufacturing': 'production processes, quality control, supply chain, industrial engineering',
            'construction': 'building projects, renovation, architecture, project management, safety',
            'automotive': 'vehicle maintenance, automotive repair, car restoration, automotive business',
            'crafts': 'handmade crafts, artisan skills, craft business, traditional techniques',
            
            # Agriculture & Food
            'agriculture': 'farming, crop management, livestock, agricultural technology, sustainable farming',
            'food': 'food production, food safety, culinary business, restaurant management, food science',
            
            # Non-Profit & Social Impact
            'nonprofit': 'nonprofit management, fundraising, volunteer coordination, social impact, community service',
            'volunteer': 'volunteer work, community service, social causes, charitable activities',
            'activism': 'social activism, campaign organizing, advocacy, community organizing, change-making',
            
            # Real Estate & Property
            'realestate': 'property investment, real estate business, property management, home buying',
            'property': 'property development, real estate marketing, property maintenance, investment',
            
            # Legal & Compliance
            'legal': 'legal research, compliance, contract management, legal documentation, law practice',
            
            # Transportation & Logistics
            'logistics': 'supply chain management, transportation, inventory, distribution, operations',
            'transportation': 'vehicle operation, route planning, transportation business, mobility solutions',
            
            # Default
            'general': 'project management, goal achievement, systematic approach, strategic planning, execution'
        }
    
    def create_smart_prompt(self, thought: str, project_type: str = 'general', complexity: str = 'moderate') -> str:
        context = self.project_contexts.get(project_type, self.project_contexts['general'])
        complexity_factor = self.complexity_weights.get(complexity, 0.6)
        
        return f"""<s>[INST] You are an expert project manager and task breakdown specialist with deep expertise in {context}. 

Analyze this project idea: "{thought}"

Project Context: {project_type.title()} project with {complexity} complexity level
Your task: Create an intelligent, comprehensive task breakdown that maximizes the chances of successful project completion.

Requirements:
1. **Strategic Analysis**: Consider the project's scope, dependencies, risks, and success factors
2. **Adaptive Complexity**: Adjust task granularity based on complexity level ({complexity})
3. **Domain Expertise**: Apply {project_type} best practices and methodologies
4. **Realistic Planning**: Provide accurate time estimates and difficulty assessments

Generate:
- Clear, measurable main goal
- Appropriate category classification  
- Priority level based on impact and urgency
- 5-8 strategically sequenced subtasks
- Project complexity score (1-10)
- Total estimated hours
- Project type classification

For each subtask provide:
- **Specific, actionable title** (avoid generic terms)
- **Detailed implementation description** 
- **Realistic time estimate** (hours/days/weeks)
- **Accurate difficulty level** (easy/medium/hard)
- **Dependencies and prerequisites**

Respond ONLY with this JSON structure:
{{
  "main_goal": "Specific, measurable goal description",
  "category": "app|business|learning|creative|lifestyle|health|productivity|other",
  "priority": "high|medium|low",
  "project_type": "{project_type}",
  "complexity_score": 1-10,
  "total_estimated_hours": total_hours_number,
  "subtasks": [
    {{
      "id": 1,
      "title": "Specific actionable task title",
      "description": "Detailed step-by-step implementation guide",
      "estimated_time": "X hours|days|weeks",
      "difficulty": "easy|medium|hard"
    }}
  ]
}}[/INST]"""

    def create_nested_prompt(self, parent_task: Dict, context: str, depth: int) -> str:
        return f"""<s>[INST] You are an expert task breakdown specialist. Break down this specific task into smaller, actionable subtasks.

Parent Task: "{parent_task['title']}"
Description: "{parent_task['description']}"
Difficulty: {parent_task['difficulty']}
Time Estimate: {parent_task['estimated_time']}

Project Context: {context}
Current Breakdown Depth: Level {depth}

Your expertise: Analyze this task and determine if it can be broken down into smaller, more manageable subtasks. Consider:
- Task complexity and scope
- Natural workflow and dependencies  
- Skill requirements and knowledge areas
- Time management and efficiency
- Quality checkpoints and milestones

If this task is complex enough to warrant breakdown (typically tasks >2 hours or medium/hard difficulty), create 3-6 specific subtasks.
If it's already sufficiently granular, return empty subtasks array.

IMPORTANT: Only break down tasks that genuinely benefit from subdivision. Don't create unnecessary micro-tasks.

Respond ONLY with JSON:
{{
  "subtasks": [
    {{
      "id": 1,
      "title": "Specific sub-task title",
      "description": "Detailed implementation steps",
      "estimated_time": "X hours|days",
      "difficulty": "easy|medium|hard"
    }}
  ],
  "breakdown_reasoning": "Brief explanation of why these subtasks improve the workflow",
  "depth_level": {depth}
}}

If no breakdown is needed, respond with:
{{
  "subtasks": [],
  "breakdown_reasoning": "Task is already sufficiently granular",
  "depth_level": {depth}
}}[/INST]"""

    def generate_smart_breakdown(self, thought: str, project_type: str = 'general', complexity: str = 'moderate') -> Dict[str, Any]:
        start_time = time.time()
        
        try:
            prompt = self.create_smart_prompt(thought, project_type, complexity)
            inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2500)
            inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = self.model.generate(**inputs, **self.generation_config)
            
            generated_ids = outputs[0][len(inputs["input_ids"][0]):]
            response = self.tokenizer.decode(generated_ids, skip_special_tokens=True)
            
            parsed_result = self.parse_smart_response(response, thought, project_type, complexity)
            processing_time = time.time() - start_time
            
            return {
                "success": True,
                "data": parsed_result,
                "processing_time": f"{processing_time:.2f}s",
                "model_response": response[:300] + "..." if len(response) > 300 else response
            }
            
        except Exception as e:
            print(f"Smart breakdown error: {str(e)}")
            return {
                "success": False,
                "error": str(e),
                "data": self.create_intelligent_fallback(thought, project_type, complexity)
            }
    
    def generate_nested_breakdown(self, parent_task: Dict, context: str, depth: int) -> Dict[str, Any]:
        start_time = time.time()
        
        try:
            if depth >= 5:
                return {
                    "success": False,
                    "error": "Maximum breakdown depth reached",
                    "subtasks": []
                }
            
            prompt = self.create_nested_prompt(parent_task, context, depth)
            inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2000)
            inputs = {k: v.to(self.model.device) for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = self.model.generate(**inputs, **self.generation_config)
            
            generated_ids = outputs[0][len(inputs["input_ids"][0]):]
            response = self.tokenizer.decode(generated_ids, skip_special_tokens=True)
            
            parsed_result = self.parse_nested_response(response, depth)
            processing_time = time.time() - start_time
            
            return {
                "success": True,
                "subtasks": parsed_result.get("subtasks", []),
                "breakdown_reasoning": parsed_result.get("breakdown_reasoning", ""),
                "depth_level": depth,
                "processing_time": f"{processing_time:.2f}s",
                "timestamp": time.time()
            }
            
        except Exception as e:
            print(f"Nested breakdown error: {str(e)}")
            return {
                "success": False,
                "error": str(e),
                "subtasks": [],
                "depth_level": depth
            }
    
    def parse_response(self, response: str, thought: str) -> Dict[str, Any]:
        try:
            start_idx = response.find('{')
            end_idx = response.rfind('}') + 1
            
            if start_idx == -1 or end_idx == 0:
                raise ValueError("No JSON found")
            
            json_str = response[start_idx:end_idx]
            parsed = json.loads(json_str)
            
            return self.validate_response(parsed, thought)
            
        except Exception:
            return self.create_fallback(thought)
    
    def validate_response(self, data: Dict[str, Any], thought: str) -> Dict[str, Any]:
        if "main_goal" not in data:
            data["main_goal"] = f"Complete project: {thought}"
        
        valid_categories = ["app", "business", "learning", "creative", "lifestyle", "health", "productivity", "other"]
        if "category" not in data or data["category"] not in valid_categories:
            data["category"] = "other"
        
        if "priority" not in data or data["priority"] not in ["high", "medium", "low"]:
            data["priority"] = "medium"
        
        if "subtasks" not in data or not isinstance(data["subtasks"], list):
            return self.create_fallback(thought)
        
        cleaned_subtasks = []
        for i, task in enumerate(data["subtasks"][:7], 1):
            cleaned_task = {
                "id": i,
                "title": str(task.get("title", f"Task {i}")).strip(),
                "description": str(task.get("description", "Complete this task")).strip(),
                "estimated_time": self.normalize_time(task.get("estimated_time", "2 hours")),
                "difficulty": task.get("difficulty", "medium") if task.get("difficulty") in ["easy", "medium", "hard"] else "medium"
            }
            cleaned_subtasks.append(cleaned_task)
        
        data["subtasks"] = cleaned_subtasks
        return data
    
    def normalize_time(self, time_str: str) -> str:
        time_str = str(time_str).lower().strip()
        
        if any(word in time_str for word in ["30", "45", "1 hour", "1hr"]):
            return "1 hour"
        elif any(word in time_str for word in ["2", "couple"]):
            return "2 hours"
        elif any(word in time_str for word in ["3", "few"]):
            return "3 hours"
        elif any(word in time_str for word in ["4", "half day"]):
            return "4 hours"
        elif any(word in time_str for word in ["day", "8 hours"]):
            return "1 day"
        elif any(word in time_str for word in ["week"]):
            return "1 week"
        else:
            return time_str if time_str else "2 hours"
    
    def create_fallback(self, thought: str) -> Dict[str, Any]:
        thought_lower = thought.lower()
        
        if any(word in thought_lower for word in ["app", "website", "software", "code"]):
            category = "app"
            tasks = [
                ("Project Planning & Requirements", "Define scope, features, and technical requirements", "3 hours", "medium"),
                ("Technology Stack Selection", "Choose frameworks, databases, and development tools", "2 hours", "easy"),
                ("UI/UX Design & Wireframes", "Create user interface mockups and user flows", "6 hours", "medium"),
                ("Development Environment Setup", "Set up dev tools, version control, and project structure", "2 hours", "easy"),
                ("Core Feature Implementation", "Build main functionality and user interface", "12 hours", "hard"),
                ("Testing & Quality Assurance", "Test features, fix bugs, ensure reliability", "4 hours", "medium")
            ]
        elif any(word in thought_lower for word in ["business", "startup", "company"]):
            category = "business"
            tasks = [
                ("Market Research & Validation", "Research target market and validate demand", "8 hours", "medium"),
                ("Business Plan Development", "Create comprehensive plan with financials", "12 hours", "hard"),
                ("Legal Structure & Registration", "Handle business registration and legal setup", "4 hours", "medium"),
                ("Brand Identity & Marketing", "Develop brand, logo, and marketing strategy", "8 hours", "medium"),
                ("Product/Service Development", "Build minimum viable product or service", "20 hours", "hard"),
                ("Customer Acquisition Strategy", "Plan and implement customer acquisition", "6 hours", "medium")
            ]
        elif any(word in thought_lower for word in ["learn", "study", "skill"]):
            category = "learning"
            tasks = [
                ("Learning Goal Definition", "Define specific objectives and success metrics", "1 hour", "easy"),
                ("Resource Research", "Find quality learning materials and courses", "3 hours", "easy"),
                ("Study Schedule Creation", "Create realistic schedule with milestones", "2 hours", "easy"),
                ("Active Learning & Practice", "Engage with material through practice", "20 hours", "medium"),
                ("Project Application", "Apply skills in real project", "8 hours", "hard"),
                ("Knowledge Assessment", "Test understanding and identify gaps", "2 hours", "medium")
            ]
        else:
            category = "other"
            tasks = [
                ("Research & Information Gathering", "Collect relevant information and understand requirements", "3 hours", "easy"),
                ("Planning & Strategy", "Create detailed plan with timeline", "4 hours", "medium"),
                ("Resource Acquisition", "Gather tools, materials, permissions", "2 hours", "easy"),
                ("Initial Implementation", "Begin main work execution", "8 hours", "medium"),
                ("Iteration & Refinement", "Review progress and improve", "4 hours", "medium"),
                ("Completion & Documentation", "Finish and document results", "3 hours", "easy")
            ]
        
        subtasks = []
        for i, (title, desc, time, diff) in enumerate(tasks[:6], 1):
            subtasks.append({
                "id": i,
                "title": title,
                "description": desc,
                "estimated_time": time,
                "difficulty": diff
            })
        
        return {
            "main_goal": f"Successfully execute: {thought}",
            "category": category,
            "priority": "medium",
            "subtasks": subtasks
        }
    
    def parse_smart_response(self, response: str, thought: str, project_type: str, complexity: str) -> Dict[str, Any]:
        try:
            start_idx = response.find('{')
            end_idx = response.rfind('}') + 1
            
            if start_idx == -1 or end_idx == 0:
                raise ValueError("No JSON found")
            
            json_str = response[start_idx:end_idx]
            parsed = json.loads(json_str)
            
            return self.validate_smart_response(parsed, thought, project_type, complexity)
            
        except Exception as e:
            print(f"Smart parsing error: {e}")
            return self.create_intelligent_fallback(thought, project_type, complexity)
    
    def parse_nested_response(self, response: str, depth: int) -> Dict[str, Any]:
        try:
            start_idx = response.find('{')
            end_idx = response.rfind('}') + 1
            
            if start_idx == -1 or end_idx == 0:
                raise ValueError("No JSON found")
            
            json_str = response[start_idx:end_idx]
            parsed = json.loads(json_str)
            
            if "subtasks" not in parsed:
                parsed["subtasks"] = []
            
            # Clean and validate subtasks
            cleaned_subtasks = []
            for i, task in enumerate(parsed["subtasks"][:6], 1):
                if isinstance(task, dict) and "title" in task:
                    cleaned_task = {
                        "id": i,
                        "title": str(task.get("title", f"Subtask {i}")).strip(),
                        "description": str(task.get("description", "Complete this subtask")).strip(),
                        "estimated_time": self.normalize_time(task.get("estimated_time", "1 hour")),
                        "difficulty": task.get("difficulty", "easy") if task.get("difficulty") in ["easy", "medium", "hard"] else "easy"
                    }
                    cleaned_subtasks.append(cleaned_task)
            
            return {
                "subtasks": cleaned_subtasks,
                "breakdown_reasoning": parsed.get("breakdown_reasoning", "Task breakdown completed"),
                "depth_level": depth
            }
            
        except Exception as e:
            print(f"Nested parsing error: {e}")
            return {
                "subtasks": [],
                "breakdown_reasoning": "Parsing failed, no subtasks generated",
                "depth_level": depth
            }
    
    def validate_smart_response(self, data: Dict[str, Any], thought: str, project_type: str, complexity: str) -> Dict[str, Any]:
        # Ensure required fields with intelligent defaults
        if "main_goal" not in data:
            data["main_goal"] = f"Successfully complete {project_type} project: {thought}"
        
        valid_categories = ["app", "business", "learning", "creative", "lifestyle", "health", "productivity", "other"]
        if "category" not in data or data["category"] not in valid_categories:
            data["category"] = project_type if project_type in valid_categories else "other"
        
        if "priority" not in data or data["priority"] not in ["high", "medium", "low"]:
            data["priority"] = "high" if complexity in ["complex", "enterprise"] else "medium"
        
        # Add missing advanced fields
        if "project_type" not in data:
            data["project_type"] = project_type
        
        if "complexity_score" not in data:
            complexity_scores = {"simple": 3, "moderate": 5, "complex": 7, "enterprise": 9}
            data["complexity_score"] = complexity_scores.get(complexity, 5)
        
        if "total_estimated_hours" not in data:
            data["total_estimated_hours"] = self.calculate_total_time(data.get("subtasks", []))
        
        if "subtasks" not in data or not isinstance(data["subtasks"], list):
            return self.create_intelligent_fallback(thought, project_type, complexity)
        
        # Enhanced subtask processing
        cleaned_subtasks = []
        for i, task in enumerate(data["subtasks"][:8], 1):  # Allow up to 8 tasks for complex projects
            cleaned_task = {
                "id": i,
                "title": str(task.get("title", f"Task {i}")).strip(),
                "description": str(task.get("description", "Complete this task")).strip(),
                "estimated_time": self.normalize_time(task.get("estimated_time", "2 hours")),
                "difficulty": task.get("difficulty", "medium") if task.get("difficulty") in ["easy", "medium", "hard"] else "medium"
            }
            cleaned_subtasks.append(cleaned_task)
        
        data["subtasks"] = cleaned_subtasks
        return data
    
    def create_intelligent_fallback(self, thought: str, project_type: str = 'general', complexity: str = 'moderate') -> Dict[str, Any]:
        thought_lower = thought.lower()
        complexity_scores = {"simple": 3, "moderate": 5, "complex": 7, "enterprise": 9}
        time_multipliers = {"simple": 0.5, "moderate": 1.0, "complex": 1.8, "enterprise": 3.0}
        
        if project_type == 'software' or any(word in thought_lower for word in ["app", "website", "software", "code"]):
            category = "app"
            base_tasks = [
                ("Requirements Analysis & Planning", "Define project scope, user stories, and technical requirements", "4 hours", "medium"),
                ("Architecture & Design", "Create system architecture and UI/UX wireframes", "6 hours", "medium"),
                ("Development Environment Setup", "Configure development tools, version control, and CI/CD", "3 hours", "easy"),
                ("Core Feature Development", "Implement main application functionality", "16 hours", "hard"),
                ("Testing & Quality Assurance", "Write tests, debug, and ensure code quality", "6 hours", "medium"),
                ("Deployment & Production Setup", "Deploy to production and configure monitoring", "4 hours", "medium"),
                ("Documentation & Handover", "Create user guides and technical documentation", "3 hours", "easy")
            ]
        elif project_type == 'business' or any(word in thought_lower for word in ["business", "startup", "company"]):
            category = "business"
            base_tasks = [
                ("Market Research & Analysis", "Analyze target market, competitors, and opportunities", "8 hours", "medium"),
                ("Business Model Development", "Define value proposition and revenue streams", "6 hours", "hard"),
                ("Financial Planning & Projections", "Create budgets, forecasts, and funding strategy", "10 hours", "hard"),
                ("Legal & Regulatory Setup", "Handle business registration and compliance", "5 hours", "medium"),
                ("Brand Development & Marketing", "Create brand identity and marketing strategy", "8 hours", "medium"),
                ("Operations & Team Building", "Establish processes and hire key team members", "12 hours", "hard"),
                ("Launch & Growth Strategy", "Execute go-to-market plan and scale operations", "15 hours", "hard")
            ]
        elif project_type == 'learning' or any(word in thought_lower for word in ["learn", "study", "skill"]):
            category = "learning"
            base_tasks = [
                ("Learning Objective Setting", "Define specific, measurable learning goals", "2 hours", "easy"),
                ("Resource Curation & Planning", "Find quality materials and create study schedule", "3 hours", "easy"),
                ("Foundation Building", "Master fundamental concepts and prerequisites", "12 hours", "medium"),
                ("Practical Application", "Apply knowledge through projects and exercises", "15 hours", "medium"),
                ("Advanced Topics & Specialization", "Dive deep into specialized areas", "20 hours", "hard"),
                ("Portfolio Development", "Create projects demonstrating mastery", "10 hours", "hard"),
                ("Continuous Practice & Improvement", "Maintain and refine skills over time", "8 hours", "medium")
            ]
        else:
            category = "other"
            base_tasks = [
                ("Project Definition & Scope", "Clearly define objectives and success criteria", "3 hours", "easy"),
                ("Research & Analysis", "Gather information and analyze requirements", "5 hours", "medium"),
                ("Planning & Strategy", "Create detailed timeline and resource allocation", "4 hours", "medium"),
                ("Initial Implementation", "Execute core project activities", "12 hours", "medium"),
                ("Review & Iteration", "Evaluate progress and make improvements", "6 hours", "medium"),
                ("Quality Assurance", "Ensure standards and requirements are met", "4 hours", "medium"),
                ("Finalization & Handover", "Complete project and document outcomes", "3 hours", "easy")
            ]
        
        # Adjust tasks based on complexity
        time_mult = time_multipliers.get(complexity, 1.0)
        task_count = {"simple": 4, "moderate": 6, "complex": 7, "enterprise": 8}.get(complexity, 6)
        
        subtasks = []
        for i, (title, desc, base_time, diff) in enumerate(base_tasks[:task_count], 1):
            # Adjust time estimates based on complexity
            base_hours = int(re.findall(r'\d+', base_time)[0]) if re.findall(r'\d+', base_time) else 2
            adjusted_hours = max(1, int(base_hours * time_mult))
            
            if adjusted_hours >= 8:
                time_str = f"{adjusted_hours // 8} day{'s' if adjusted_hours // 8 > 1 else ''}"
            else:
                time_str = f"{adjusted_hours} hour{'s' if adjusted_hours > 1 else ''}"
            
            subtasks.append({
                "id": i,
                "title": title,
                "description": desc,
                "estimated_time": time_str,
                "difficulty": diff
            })
        
        total_hours = sum(int(re.findall(r'\d+', task["estimated_time"])[0]) for task in subtasks)
        
        return {
            "main_goal": f"Successfully execute {complexity} {project_type} project: {thought}",
            "category": category,
            "priority": "high" if complexity in ["complex", "enterprise"] else "medium",
            "project_type": project_type,
            "complexity_score": complexity_scores.get(complexity, 5),
            "total_estimated_hours": total_hours,
            "subtasks": subtasks
        }
    
    def calculate_total_time(self, subtasks: List[Dict]) -> int:
        total = 0
        for task in subtasks:
            time_str = task.get("estimated_time", "1 hour").lower()
            numbers = re.findall(r'\d+', time_str)
            if numbers:
                value = int(numbers[0])
                if 'day' in time_str:
                    total += value * 8
                elif 'week' in time_str:
                    total += value * 40
                else:  # assume hours
                    total += value
        return total

# Initialize Enhanced AI system
print("🧠 Initializing Mistral Infinite Task Breakdown AI...")
task_ai = MistralInfiniteTaskBreakdownAI(model, tokenizer)
print("✅ Advanced AI system ready for infinite task breakdown!")

🧠 Initializing Mistral Infinite Task Breakdown AI...


NameError: name 'model' is not defined

In [ ]:
# 🚀 FAST Test - Optimized for Kaggle Speed
print("⚡ FAST TEST: Quick Mistral 7B Validation")
print("="*50)

# Single fast test
test_thought = "Create a simple mobile app"
complexity = "simple"
project_type = "software"

print(f"🎯 Testing: {test_thought}")
print(f"🔧 Complexity: {complexity.upper()}")
print(f"📱 Type: {project_type.upper()}")
print("🕐 Starting generation (should be ~30-60 seconds)...")

start_time = time.time()
result = task_ai.generate_smart_breakdown(test_thought, project_type, complexity)
total_time = time.time() - start_time

if result["success"]:
    data = result["data"]
    print(f"\n✅ SUCCESS in {total_time:.1f} seconds!")
    print(f"🎯 Goal: {data['main_goal']}")
    print(f"📊 Subtasks: {len(data['subtasks'])}")
    print(f"⏱️ Total Hours: {data.get('total_estimated_hours', 'N/A')}")
    
    print(f"\n📋 Generated Tasks:")
    for i, task in enumerate(data['subtasks'][:3], 1):  # Show first 3
        print(f"  {i}. {task['title']}")
        print(f"     ⏱️ {task['estimated_time']} | 🎚️ {task['difficulty']}")
    
    if len(data['subtasks']) > 3:
        print(f"  ... and {len(data['subtasks']) - 3} more tasks")
    
    print(f"\n🎉 Mistral 7B is working perfectly!")
    print(f"🚀 Ready for full testing!")
    
else:
    print(f"❌ Error: {result.get('error')}")
    print("🔄 Check your model loading and try again")

print("\n" + "="*50)


Test Enhanced System

In [ ]:
test_thoughts = [
    "Create a cryptocurrency trading platform with AI predictions",
    "Launch a sustainable fashion e-commerce marketplace", 
    "Build a machine learning course for beginners",
    "Design a smart city IoT infrastructure project"
]

complexities = ['simple', 'moderate', 'complex', 'enterprise']

print("🧪 Testing Advanced Mistral 7B Infinite Breakdown System...")
print("="*80)

for i, thought in enumerate(test_thoughts, 1):
    complexity = complexities[(i-1) % len(complexities)]
    
    print(f"\n🎯 TEST {i}: {thought}")
    print(f"🔧 Complexity: {complexity.upper()}")
    print("-" * 60)
    
    # Detect project type
    thought_lower = thought.lower()
    if 'platform' in thought_lower or 'app' in thought_lower or 'trading' in thought_lower:
        project_type = 'software'
    elif 'launch' in thought_lower or 'marketplace' in thought_lower or 'business' in thought_lower:
        project_type = 'business'
    elif 'course' in thought_lower or 'learn' in thought_lower:
        project_type = 'learning'
    else:
        project_type = 'general'
    
    result = task_ai.generate_smart_breakdown(thought, project_type, complexity)
    
    if result["success"]:
        data = result["data"]
        print(f"🎯 MAIN GOAL: {data['main_goal']}")
        print(f"📂 PROJECT TYPE: {data.get('project_type', 'N/A').upper()}")
        print(f"⚡ PRIORITY: {data['priority'].upper()}")
        print(f"🔥 COMPLEXITY: {data.get('complexity_score', 'N/A')}/10")
        print(f"⏱️ TOTAL TIME: {data.get('total_estimated_hours', 'N/A')} hours")
        print(f"🚀 PROCESSING: {result.get('processing_time', 'N/A')}")
        print(f"\n📋 INTELLIGENT SUBTASKS ({len(data['subtasks'])}):")
        
        for task in data['subtasks']:
            breakdown_indicator = "🔄" if (
                task['difficulty'] != 'easy' and 
                ('hour' in task['estimated_time'] and int(re.findall(r'\d+', task['estimated_time'])[0]) > 2 or
                 'day' in task['estimated_time'] or 'week' in task['estimated_time'])
            ) else "✅"
            
            print(f"\n  {breakdown_indicator} {task['id']}. {task['title']}")
            print(f"     📝 {task['description']}")
            print(f"     ⏱️ {task['estimated_time']} | 🎚️ {task['difficulty'].upper()}")
        
        print(f"\n📊 PROJECT INSIGHTS:")
        print(f"   • Can breakdown: {sum(1 for t in data['subtasks'] if t['difficulty'] != 'easy')} tasks")
        print(f"   • Complexity distribution: {[t['difficulty'] for t in data['subtasks']]}")
        print(f"   • Ready for infinite nested breakdown!")
        
        # Test nested breakdown on first complex task
        complex_tasks = [t for t in data['subtasks'] if t['difficulty'] in ['medium', 'hard']]
        if complex_tasks:
            print(f"\n🔬 TESTING NESTED BREAKDOWN ON: {complex_tasks[0]['title']}")
            nested_result = task_ai.generate_nested_breakdown(
                complex_tasks[0], 
                data['main_goal'], 
                depth=1
            )
            
            if nested_result["success"] and nested_result["subtasks"]:
                print(f"   ✅ Generated {len(nested_result['subtasks'])} sub-subtasks!")
                print(f"   💡 Reasoning: {nested_result.get('breakdown_reasoning', 'N/A')}")
                for subtask in nested_result["subtasks"][:3]:  # Show first 3
                    print(f"      → {subtask['title']} ({subtask['estimated_time']})")
            else:
                print(f"   ℹ️  Task doesn't need further breakdown")
        
    else:
        print(f"❌ Error: {result.get('error')}")
    
    print("\n" + "="*80)

print("\n🎉 ADVANCED TESTING COMPLETED!")
print("🚀 Mistral 7B now provides:")
print("   • Context-aware project analysis")
print("   • Adaptive complexity handling") 
print("   • Infinite nested task breakdown")
print("   • Domain-specific expertise")
print("   • Realistic time estimation")

Enhanced Flask API Server

In [ ]:
import threading
import time
from flask import Flask, request, jsonify
from datetime import datetime

# 🌐 Setup Ngrok Tunnel & Start Flask Server
print("🌐 Setting up public URL tunnel...")

# Start Flask app in background thread
def run_flask():
    app.run(host='0.0.0.0', port=5000, threaded=True, debug=False)

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()
print("🚀 Flask server starting...")

# Wait for Flask to start
time.sleep(5)

# Create ngrok tunnel
try:
    print("🔗 Creating ngrok tunnel...")
    public_tunnel = ngrok.connect(5000)
    public_url = public_tunnel.public_url
    print(f"✅ Ngrok tunnel created successfully!")
    print(f"🌍 Public URL: {public_url}")
    
    # Display connection info
    print("\n" + "="*70)
    print("🎉 SHOWERLOG AI SERVER IS LIVE!")
    print("="*70)
    print(f"🌐 Public URL: {public_url}")
    print(f"🔗 Add this to your .env.local:")
    print(f"   NEXT_PUBLIC_AI_API_URL={public_url}")
    print("="*70)
    
except Exception as e:
    print(f"❌ Ngrok tunnel failed: {e}")
    print("🔄 Server running locally only")
    public_url = "http://localhost:5000"
    print(f"🏠 Local URL: {public_url}")

print(f"\n🚀 Available endpoints:")
print(f"   • {public_url}/breakdown-smart - Intelligent task breakdown")
print(f"   • {public_url}/breakdown-nested - Nested task breakdown") 
print(f"   • {public_url}/generate-thoughts - Random thought generation")
print(f"   • {public_url}/health - Server health check")
print(f"   • {public_url}/stats - Server statistics")
print(f"   • {public_url}/keepalive - Keep-alive endpoint")


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from datetime import datetime
import threading
import time
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = Flask(__name__)
CORS(app, origins=["*"])

# Global state
request_count = 0
breakdown_count = 0
nested_breakdown_count = 0
start_time = datetime.now()

@app.route('/health', methods=['GET'])
def health_check():
    global request_count
    request_count += 1
    uptime = datetime.now() - start_time
    
    return jsonify({
        "status": "healthy",
        "model": "mistralai/Mistral-7B-Instruct-v0.2",
        "version": "infinite-breakdown-v2.0",
        "uptime": str(uptime).split('.')[0],
        "requests_served": request_count,
        "breakdowns_generated": breakdown_count,
        "nested_breakdowns": nested_breakdown_count,
        "gpu_available": torch.cuda.is_available(),
        "capabilities": [
            "smart_breakdown",
            "nested_breakdown", 
            "infinite_recursion",
            "complexity_adaptation",
            "domain_expertise"
        ],
        "timestamp": datetime.now().isoformat()
    })

@app.route('/breakdown-smart', methods=['POST'])
def smart_breakdown():
    global request_count, breakdown_count
    request_count += 1
    start_time_req = time.time()
    
    try:
        data = request.get_json()
        
        if not data or 'thought' not in data:
            return jsonify({
                "error": "Missing 'thought' field",
                "success": False,
                "example": {
                    "thought": "Create a mobile app",
                    "project_type": "software",
                    "complexity_level": "moderate"
                }
            }), 400
        
        thought = data['thought'].strip()
        project_type = data.get('project_type', 'general')
        complexity = data.get('complexity_level', 'moderate')
        
        if len(thought) < 10:
            return jsonify({
                "error": "Thought too short (min 10 chars)",
                "success": False
            }), 400
        
        logger.info(f"Smart breakdown: {thought[:50]}... [Type: {project_type}, Complexity: {complexity}]")
        
        result = task_ai.generate_smart_breakdown(thought, project_type, complexity)
        breakdown_count += 1
        
        processing_time = time.time() - start_time_req
        
        if result["success"]:
            response_data = {
                "success": True,
                "data": result["data"],
                "processing_time": f"{processing_time:.2f}s",
                "model": "mistralai/Mistral-7B-Instruct-v0.2",
                "breakdown_type": "smart",
                "project_analysis": {
                    "detected_type": project_type,
                    "complexity_level": complexity,
                    "total_tasks": len(result["data"]["subtasks"]),
                    "breakdownable_tasks": sum(1 for t in result["data"]["subtasks"] 
                                             if t["difficulty"] != "easy"),
                },
                "timestamp": datetime.now().isoformat()
            }
            
            logger.info(f"Smart breakdown successful: {len(result['data']['subtasks'])} tasks, {processing_time:.2f}s")
            return jsonify(response_data)
        else:
            logger.error(f"Smart breakdown failed: {result.get('error')}")
            return jsonify({
                "success": False,
                "error": result.get("error", "Unknown error"),
                "fallback_data": result.get("data"),
                "processing_time": f"{processing_time:.2f}s",
                "timestamp": datetime.now().isoformat()
            }), 500
        
    except Exception as e:
        logger.error(f"Smart breakdown endpoint error: {str(e)}")
        return jsonify({
            "error": f"Server error: {str(e)}",
            "success": False,
            "timestamp": datetime.now().isoformat()
        }), 500

@app.route('/breakdown-nested', methods=['POST'])
def nested_breakdown():
    global request_count, nested_breakdown_count
    request_count += 1
    nested_breakdown_count += 1
    start_time_req = time.time()
    
    try:
        data = request.get_json()
        
        required_fields = ['parent_task', 'context', 'depth']
        if not data or not all(field in data for field in required_fields):
            return jsonify({
                "error": f"Missing required fields: {required_fields}",
                "success": False,
                "example": {
                    "parent_task": {
                        "title": "Task title",
                        "description": "Task description", 
                        "difficulty": "medium",
                        "estimated_time": "4 hours"
                    },
                    "context": "Project context",
                    "depth": 1,
                    "max_depth": 5
                }
            }), 400
        
        parent_task = data['parent_task']
        context = data['context']
        depth = data['depth']
        max_depth = data.get('max_depth', 5)
        
        if depth >= max_depth:
            return jsonify({
                "success": False,
                "error": f"Maximum depth ({max_depth}) reached",
                "subtasks": [],
                "depth_level": depth
            })
        
        logger.info(f"Nested breakdown: {parent_task.get('title', 'Unknown')[:30]}... [Depth: {depth}]")
        
        result = task_ai.generate_nested_breakdown(parent_task, context, depth)
        
        processing_time = time.time() - start_time_req
        
        response_data = {
            "success": result["success"],
            "subtasks": result.get("subtasks", []),
            "breakdown_reasoning": result.get("breakdown_reasoning", ""),
            "depth_level": depth,
            "processing_time": f"{processing_time:.2f}s",
            "model": "mistralai/Mistral-7B-Instruct-v0.2",
            "breakdown_type": "nested",
            "parent_task_title": parent_task.get('title', 'Unknown'),
            "generated_subtasks": len(result.get("subtasks", [])),
            "timestamp": datetime.now().isoformat()
        }
        
        if result["success"]:
            logger.info(f"Nested breakdown successful: {len(result.get('subtasks', []))} subtasks at depth {depth}")
        else:
            logger.warning(f"Nested breakdown returned no tasks: {result.get('error', 'No reason given')}")
        
        return jsonify(response_data)
        
    except Exception as e:
        logger.error(f"Nested breakdown endpoint error: {str(e)}")
        return jsonify({
            "error": f"Server error: {str(e)}",
            "success": False,
            "subtasks": [],
            "depth_level": data.get('depth', 0),
            "timestamp": datetime.now().isoformat()
        }), 500

@app.route('/breakdown', methods=['POST'])
def legacy_breakdown():
    """Legacy endpoint for backwards compatibility"""
    data = request.get_json()
    
    # Convert to smart breakdown
    if data and 'thought' in data:
        data['project_type'] = 'general'
        data['complexity_level'] = 'moderate'
        
        # Forward to smart breakdown
        request.json = data
        return smart_breakdown()
    else:
        return jsonify({
            "error": "Missing 'thought' field",
            "success": False
        }), 400

@app.route('/generate-thoughts', methods=['GET'])
def generate_thoughts():
    global request_count
    request_count += 1
    
    thoughts_by_category = {
        # Technology & Development
        "software": [
            "Build a simple to-do app for my daily tasks",
            "Create a personal website to showcase my hobbies",
            "Learn to code and build my first project",
            "Make a calculator app with a nice design"
        ],
        "web": [
            "Create a blog about my interests and experiences",
            "Build a simple portfolio website for myself",
            "Make a landing page for a small business idea",
            "Design a website for my local community group"
        ],
        "mobile": [
            "Create a simple habit tracking app",
            "Build a grocery list app for my family",
            "Make a photo journal app for daily memories",
            "Design a water drinking reminder app"
        ],
        "ai": [
            "Learn the basics of artificial intelligence",
            "Understand how recommendation systems work",
            "Explore simple machine learning concepts",
            "Try out AI tools for everyday tasks"
        ],
        
        # Business & Finance
        "business": [
            "Start a small online business selling handmade items",
            "Create a budget plan for my monthly expenses",
            "Learn basic investing with small amounts",
            "Plan a garage sale to declutter and earn money"
        ],
        "startup": [
            "Turn my hobby into a side income",
            "Start a pet-sitting service in my neighborhood",
            "Create a simple service to help busy families",
            "Launch a small tutoring business for local students"
        ],
        "finance": [
            "Create an emergency fund with $1000",
            "Learn to track my spending better",
            "Plan to pay off my credit card debt",
            "Start saving for a vacation next year"
        ],
        "marketing": [
            "Promote my small business on social media",
            "Help a friend advertise their garage sale",
            "Create content to share my expertise online",
            "Start a simple newsletter for my community"
        ],
        
        # Creative & Arts
        "creative": [
            "Start a daily drawing or sketching practice",
            "Write short stories about my daily experiences",
            "Create handmade gifts for family and friends",
            "Document my life through creative photography"
        ],
        "art": [
            "Learn watercolor painting as a relaxing hobby",
            "Create art using recycled materials from home",
            "Start a small art journal with daily sketches",
            "Make decorative pieces for my living space"
        ],
        "music": [
            "Learn to play my favorite song on guitar",
            "Create simple beats using free music software",
            "Start a small podcast about topics I love",
            "Learn basic ukulele to play around campfires"
        ],
        "photography": [
            "Take better photos of my family gatherings",
            "Document my neighborhood's changing seasons",
            "Create a photo album of local coffee shops",
            "Learn smartphone photography techniques"
        ],
        
        # Health & Wellness
        "fitness": [
            "Start walking 30 minutes every day",
            "Learn basic yoga poses for morning stretches",
            "Plan workouts I can do at home without equipment",
            "Train to walk/run a local 5K event"
        ],
        "nutrition": [
            "Learn to meal prep for busy weekdays",
            "Try cooking one new healthy recipe each week",
            "Start growing herbs on my windowsill",
            "Plan balanced meals on a budget"
        ],
        "beauty": [
            "Develop a simple skincare routine that works",
            "Learn basic makeup techniques for everyday wear",
            "Try DIY face masks using kitchen ingredients",
            "Create a capsule wardrobe with pieces I love"
        ],
        "mental_health": [
            "Start a 5-minute daily meditation practice",
            "Keep a gratitude journal for better mood",
            "Learn stress management techniques for work",
            "Create a calming bedtime routine"
        ],
        
        # Education & Learning
        "learning": [
            "Learn a new language using free apps",
            "Take an online course about something I'm curious about",
            "Read one book per month on topics I enjoy",
            "Practice a skill for 15 minutes daily"
        ],
        "language": [
            "Learn basic Spanish for my next vacation",
            "Practice English conversation with language partners",
            "Learn common phrases in my neighbor's language",
            "Understand the basics of sign language"
        ],
        "teaching": [
            "Teach my kids a skill I know well",
            "Share my expertise with friends who are interested",
            "Create simple tutorials for things I've learned",
            "Mentor someone who's just starting in my field"
        ],
        
        # Sports & Recreation  
        "sports": [
            "Join a local recreational sports league",
            "Learn to ride a bike or improve my cycling",
            "Try a new sport like tennis or badminton",
            "Organize neighborhood games for kids and families"
        ],
        "dancing": [
            "Learn basic dance moves for social events",
            "Try online dance tutorials for fun exercise",
            "Take a beginner's class in a style I like",
            "Practice dancing to my favorite songs at home"
        ],
        "outdoor": [
            "Explore hiking trails in my local area",
            "Plan fun camping trips with family or friends",
            "Create a backyard garden space for relaxation",
            "Try geocaching or nature scavenger hunts"
        ],
        
        # Personal Development & Relationships
        "relationships": [
            "Plan more quality time with family and friends",
            "Learn better communication skills for work and home",
            "Organize regular gatherings with people I care about",
            "Practice being a better listener in conversations"
        ],
        "personal": [
            "Create a morning routine that energizes me",
            "Set and achieve small personal goals each month",
            "Learn to say no to commitments that drain me",
            "Develop confidence in public speaking"
        ],
        "parenting": [
            "Plan fun educational activities for my children",
            "Create family traditions that bring us closer",
            "Learn positive discipline techniques that work",
            "Organize playdates and social activities for kids"
        ],
        
        # Home & Lifestyle
        "cooking": [
            "Master 5 simple, delicious weeknight dinners",
            "Learn to bake bread or cookies from scratch",
            "Try cooking cuisines from different cultures",
            "Organize my kitchen for easier meal preparation"
        ],
        "gardening": [
            "Start a small vegetable garden in my backyard",
            "Learn to care for houseplants successfully",
            "Create a flower garden to attract butterflies",
            "Grow fresh herbs for cooking on my balcony"
        ],
        "home": [
            "Organize and declutter one room at a time",
            "Redecorate my living space on a budget",
            "Learn basic home maintenance skills",
            "Create cozy spaces for reading and relaxation"
        ],
        
        # Career & Professional
        "career": [
            "Update my resume and LinkedIn profile",
            "Learn a new skill that could help at work",
            "Network with colleagues in my field",
            "Set professional goals for the next year"
        ],
        "job_hunting": [
            "Prepare for job interviews with practice questions",
            "Research companies I'd like to work for",
            "Improve my professional online presence",
            "Practice explaining my experience clearly"
        ],
        
        # Science & Environment
        "science": [
            "Learn about local wildlife and ecosystems",
            "Start composting kitchen scraps at home",
            "Understand basic astronomy and stargazing",
            "Explore simple science experiments with kids"
        ],
        "environment": [
            "Reduce plastic use in my daily routine",
            "Start recycling more effectively at home",
            "Choose eco-friendly alternatives for cleaning",
            "Participate in local community cleanup events"
        ],
        
        # Unique & Diverse Categories
        "travel": [
            "Plan affordable weekend trips to nearby cities",
            "Explore interesting places in my own city",
            "Learn about different cultures through food and events",
            "Create a travel savings plan for future adventures"
        ],
        "volunteer": [
            "Help at a local food bank or charity",
            "Volunteer at community events and festivals",
            "Assist elderly neighbors with simple tasks",
            "Participate in local school or library programs"
        ],
        "hobby": [
            "Learn a craft like knitting, woodworking, or pottery",
            "Start collecting something interesting and affordable",
            "Join a local club related to my interests",
            "Try different hobbies to find what I enjoy most"
        ]
    }
    
    import random
    
    # Select diverse thoughts from different categories
    selected = []
    categories = list(thoughts_by_category.keys())
    
    for _ in range(3):
        category = random.choice(categories)
        thought = random.choice(thoughts_by_category[category])
        selected.append({
            "thought": thought,
            "suggested_type": category,
            "suggested_complexity": random.choice(["moderate", "complex"])
        })
    
    return jsonify({
        "success": True,
        "thoughts": [item["thought"] for item in selected],
        "enhanced_suggestions": selected,
        "categories": categories,
        "total_available": sum(len(thoughts) for thoughts in thoughts_by_category.values()),
        "timestamp": datetime.now().isoformat()
    })

@app.route('/stats', methods=['GET'])
def get_stats():
    uptime = datetime.now() - start_time
    
    return jsonify({
        "uptime": str(uptime).split('.')[0],
        "requests_served": request_count,
        "breakdowns_generated": breakdown_count,
        "nested_breakdowns": nested_breakdown_count,
        "model": "mistralai/Mistral-7B-Instruct-v0.2",
        "version": "infinite-breakdown-v2.0",
        "gpu_memory_gb": torch.cuda.get_device_properties(0).total_memory / 1024**3 if torch.cuda.is_available() else None,
        "status": "operational",
        "capabilities": {
            "smart_breakdown": True,
            "nested_breakdown": True,
            "infinite_recursion": True,
            "complexity_adaptation": True,
            "domain_expertise": True,
            "max_depth": 5
        },
        "timestamp": datetime.now().isoformat()
    })

# Add keep-alive endpoint for external monitoring
@app.route('/keepalive')
def keepalive_endpoint():
    """Dedicated endpoint for external keep-alive pings"""
    return jsonify({
        'status': 'alive',
        'uptime': str(datetime.now() - start_time).split('.')[0],
        'requests': request_count,
        'breakdown_count': breakdown_count,
        'memory_mb': torch.cuda.memory_allocated() / 1024**2 if torch.cuda.is_available() else 0,
        'timestamp': datetime.now().isoformat(),
        'model_loaded': 'model' in globals(),
        'gpu_available': torch.cuda.is_available()
    })


print("✅ Enhanced Flask API server ready!")
print("🚀 Advanced Endpoints available:")
print("   • /breakdown-smart (POST) - Intelligent task breakdown")
print("   • /breakdown-nested (POST) - Infinite nested breakdown")
print("   • /breakdown (POST) - Legacy compatibility")
print("   • /generate-thoughts (GET) - Enhanced thought generation")
print("   • /health (GET) - Health check with capabilities")
print("   • /stats (GET) - Detailed server statistics")
print("   • /keepalive (GET) - Keep-alive monitoring endpoint")

In [ ]:
# 🔧 EMERGENCY FIX: Ensure public_url is defined
try:
    # Check if public_url exists from ngrok setup
    if 'public_url' not in globals():
        print("⚠️ Public URL not defined, setting up now...")
        
        # Try to get ngrok tunnel info
        try:
            tunnels = ngrok.get_tunnels()
            if tunnels:
                public_url = tunnels[0].public_url
                print(f"✅ Found existing tunnel: {public_url}")
            else:
                print("🔗 Creating new ngrok tunnel...")
                tunnel = ngrok.connect(5000)
                public_url = tunnel.public_url
                print(f"✅ New tunnel created: {public_url}")
        except Exception as ngrok_error:
            print(f"❌ Ngrok failed: {ngrok_error}")
            public_url = "http://localhost:5000"
            print(f"🏠 Using local URL: {public_url}")
    
    # Display the critical information
    print("\n" + "🌟" * 70)
    print("🎉 SHOWERLOG AI SERVER IS READY!")
    print("🌟" * 70)
    print(f"🌐 Public URL: {public_url}")
    print(f"🧠 Model: Mistral-7B-Instruct (Kaggle Optimized)")
    print(f"🚀 Status: Ready for requests")
    print("\n🎯 IMPORTANT: Add this to your .env.local file:")
    print(f"NEXT_PUBLIC_AI_API_URL={public_url}")
    print("\n📱 Test endpoints:")
    print(f"  • Health: {public_url}/health")
    print(f"  • Stats: {public_url}/stats") 
    print(f"  • Keep-alive: {public_url}/keepalive")
    print("🌟" * 70)
    
except Exception as e:
    print(f"🚨 Setup error: {e}")
    public_url = "http://localhost:5000"
    print(f"🏠 Fallback URL: {public_url}")
    print(f"🔧 Add to .env.local: NEXT_PUBLIC_AI_API_URL={public_url}")


Advanced Timeout Prevention System

In [ ]:
# Enhanced Keep-Alive with Aggressive Timeout Prevention
import time
import threading
from datetime import datetime, timedelta
import json
import gc
import os
import random
import psutil

class KaggleKeepAlive:
    def __init__(self):
        self.start_time = datetime.now()
        self.last_activity = datetime.now()
        self.is_running = True
        self.request_count = 0
        self.activity_count = 0
        self.stats = {
            'uptime': '0:00:00',
            'activities_performed': 0,
            'last_activity': 'None',
            'memory_usage': '0 MB',
            'gpu_usage': '0 MB',
            'model_status': 'Ready',
            'cpu_activity': 0,
            'timeout_prevention': 'Active',
            'threads_running': 0
        }
        
    def aggressive_activity_simulation(self):
        """Multiple activity strategies to prevent timeout"""
        while self.is_running:
            try:
                current_time = datetime.now()
                uptime = current_time - self.start_time
                self.stats['uptime'] = str(uptime).split('.')[0]
                
                # Activity 1: CPU computation
                cpu_work = sum(i**2 for i in range(500 + random.randint(0, 500)))
                self.stats['cpu_activity'] = cpu_work % 1000
                
                # Activity 2: Memory operations
                temp_data = [random.random() for _ in range(1000)]
                processed = [x * 2 for x in temp_data[:500]]
                self.stats['memory_usage'] = f"{psutil.virtual_memory().used / (1024**2):.1f} MB"
                del temp_data, processed
                
                # Activity 3: GPU memory check (if available)
                if torch.cuda.is_available():
                    gpu_mem = torch.cuda.memory_allocated() / (1024**2)
                    self.stats['gpu_usage'] = f"{gpu_mem:.1f} MB"
                    # Touch GPU memory
                    dummy_tensor = torch.randn(100, 100).cuda()
                    result = dummy_tensor.sum()
                    del dummy_tensor
                
                # Activity 4: File I/O operations
                temp_file = f'/tmp/keepalive_{random.randint(1000, 9999)}.txt'
                with open(temp_file, 'w') as f:
                    f.write(f"Active at {current_time}\n{random.random()}")
                if os.path.exists(temp_file):
                    with open(temp_file, 'r') as f:
                        content = f.read()
                    os.remove(temp_file)
                
                # Activity 5: Model interaction every 8 minutes
                if current_time.minute % 8 == 0 and current_time.second < 10:
                    try:
                        print("🔥 Model warmup - preventing timeout...")
                        warmup_thought = ai_service.generate_random_thought()
                        print(f"💭 Warmup: {warmup_thought[:50]}...")
                        self.stats['model_status'] = 'Warmed'
                    except Exception as e:
                        print(f"⚠️ Warmup warning: {e}")
                        self.stats['model_status'] = f'Warning: {str(e)[:20]}'
                
                # Activity 6: Garbage collection (every 15 minutes)
                if current_time.minute % 15 == 0 and current_time.second < 5:
                    gc.collect()
                    if torch.cuda.is_available():
                        torch.cuda.empty_cache()
                    print("🧹 Memory cleanup completed")
                
                # Activity 7: Network simulation
                try:
                    import requests
                    response = requests.get('https://httpbin.org/uuid', timeout=3)
                    if response.status_code == 200:
                        uuid_data = response.json()
                except:
                    pass  # Ignore network errors
                
                self.activity_count += 1
                self.stats['activities_performed'] = self.activity_count
                self.stats['last_activity'] = current_time.strftime('%H:%M:%S')
                
                # Status display every 2 minutes
                if current_time.minute % 2 == 0 and current_time.second < 10:
                    self.display_status()
                
                # Variable sleep to appear more natural
                sleep_time = random.uniform(20, 40)  # 20-40 seconds
                time.sleep(sleep_time)
                
            except Exception as e:
                print(f"⚠️ Activity simulation error: {e}")
                time.sleep(30)
    
    def emergency_prevention(self):
        """Emergency timeout prevention with micro-activities"""
        while self.is_running:
            try:
                # Continuous micro-activities every 10 seconds
                for i in range(60):  # 10 minutes of micro-activities
                    if not self.is_running:
                        break
                    
                    # Micro CPU work
                    result = sum(range(random.randint(50, 200)))
                    
                    # Memory micro-allocation
                    temp_list = [random.random() for _ in range(random.randint(10, 50))]
                    temp_sum = sum(temp_list)
                    del temp_list
                    
                    # Random mathematical operations
                    math_result = random.random() ** 0.5 * random.randint(1, 100)
                    
                    time.sleep(10)  # 10-second intervals
                    
            except Exception as e:
                print(f"Emergency prevention error: {e}")
                time.sleep(60)
    
    def model_interaction_loop(self):
        """Dedicated thread for model interactions"""
        while self.is_running:
            try:
                time.sleep(300)  # Every 5 minutes
                if self.is_running:
                    # Test different model capabilities
                    test_cases = [
                        ("organize my desk", "simple"),
                        ("learn a new skill", "moderate"),
                        ("start a small business", "complex")
                    ]
                    
                    test_case = random.choice(test_cases)
                    thought, complexity = test_case
                    
                    # Generate breakdown to keep model active
                    result = ai_service.generate_task_breakdown(thought, complexity, max_subtasks=3)
                    
                    if result and 'subtasks' in result:
                        subtask_count = len(result.get('subtasks', []))
                        print(f"🧠 Model interaction successful: {subtask_count} subtasks for '{thought}'")
                        self.stats['model_status'] = f'Active ({subtask_count} tasks)'
                    else:
                        print("🧠 Model interaction completed (fallback used)")
                        self.stats['model_status'] = 'Active (fallback)'
                        
            except Exception as e:
                print(f"Model interaction error: {e}")
                self.stats['model_status'] = f'Error: {str(e)[:20]}'
                time.sleep(300)
    
    def display_status(self):
        """Display comprehensive status"""
        current_time = datetime.now()
        print("\n" + "🚿" * 25)
        print(f"🚿 ShowerThoughts AI Server Status")
        print(f"🚿 {current_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print("🚿" * 25)
        print(f"⏰ Uptime: {self.stats['uptime']}")
        print(f"🔥 Activities: {self.stats['activities_performed']}")
        print(f"📊 API Requests: {request_count}")
        print(f"💾 Memory: {self.stats['memory_usage']}")
        print(f"🎮 GPU Memory: {self.stats['gpu_usage']}")
        print(f"🤖 Model: {self.stats['model_status']}")
        print(f"⚡ CPU Work: {self.stats['cpu_activity']}")
        print(f"🛡️ Prevention: {self.stats['timeout_prevention']}")
        print(f"🕐 Last Activity: {self.stats['last_activity']}")
        print(f"🧵 Threads: {threading.active_count()}")
        print("🚿" * 25 + "\n")
    
    def update_activity(self):
        """Update activity with enhanced tracking"""
        self.last_activity = datetime.now()
        self.request_count += 1
        self.stats['model_status'] = 'Request Active'
        
        # Burst activity after requests
        burst_work = sum(i**3 for i in range(100))
        temp_data = [random.random() for _ in range(500)]
        del temp_data
        
    def stop(self):
        """Stop all keep-alive services"""
        self.is_running = False
        print("🛑 All keep-alive services stopped")

# Initialize enhanced keep-alive system
print("🎯 Starting Comprehensive Timeout Prevention System...")
keep_alive = KaggleKeepAlive()

# Start multiple redundant threads
print("🚀 Launching multiple protection threads...")

# Thread 1: Aggressive activity simulation
activity_thread = threading.Thread(target=keep_alive.aggressive_activity_simulation, daemon=True)
activity_thread.start()
print("✅ Activity simulation thread started")

# Thread 2: Emergency prevention
emergency_thread = threading.Thread(target=keep_alive.emergency_prevention, daemon=True)
emergency_thread.start()
print("✅ Emergency prevention thread started")

# Thread 3: Model interactions
model_thread = threading.Thread(target=keep_alive.model_interaction_loop, daemon=True)
model_thread.start()
print("✅ Model interaction thread started")

# Thread 4: Additional protection layer
def extra_protection():
    while keep_alive.is_running:
        try:
            # Random computational tasks
            for _ in range(10):
                work = [i * random.random() for i in range(random.randint(100, 500))]
                result = sum(work) / len(work) if work else 0
                del work
                time.sleep(random.uniform(30, 60))
        except Exception as e:
            print(f"Extra protection error: {e}")
            time.sleep(60)

extra_thread = threading.Thread(target=extra_protection, daemon=True)
extra_thread.start()
print("✅ Extra protection thread started")

# Update Flask app to track activity
@app.before_request
def before_request():
    """Track all requests for activity monitoring"""
    keep_alive.update_activity()

# Display system readiness
print("\n" + "="*70)
print("🚿 SHOWERTHOUGHTS AI SERVER - FULLY PROTECTED!")
print("="*70)
print(f"🌐 Public URL: {public_url}")
print(f"🧠 Model: Mistral-7B-Instruct (4-bit optimized)")
print(f"💾 Memory: {keep_alive.stats['memory_usage']}")
print(f"🎲 Random Thoughts: 50+ accessible categories")
print(f"🔄 Infinite Breakdown: Up to 5 levels deep")
print(f"🛡️ Timeout Prevention: {threading.active_count()} threads active")
print(f"📊 Keep-Alive Endpoint: {public_url}/keepalive")
print(f"🔥 Activities Running: {keep_alive.stats['activities_performed']}")
print("="*70)

# Enhanced main loop with maximum activity
print("🎮 Starting maximum activity main loop...")
iteration_count = 0

try:
    while True:
        iteration_count += 1
        current_time = datetime.now()
        
        # Intense activity burst every iteration
        activity_burst = {
            'iteration': iteration_count,
            'timestamp': current_time.isoformat(),
            'random_work': sum(random.randint(1, 1000) for _ in range(100)),
            'memory_work': [random.random() for _ in range(random.randint(500, 1500))]
        }
        
        # Mathematical computations
        math_intensive = sum(i**2.5 for i in range(1, random.randint(100, 500)))
        
        # Memory allocation and deallocation
        large_data = list(range(random.randint(5000, 15000)))
        processed = [x * random.random() for x in large_data[:random.randint(1000, 3000)]]
        del large_data, processed
        
        # Periodic mega-activities
        if iteration_count % 15 == 0:  # Every ~2.5 minutes
            print(f"🚀 MEGA ACTIVITY BURST #{iteration_count // 15}")
            
            # CPU mega-task
            mega_calc = sum(i**3 for i in range(2000))
            
            # Memory mega-operations
            mega_data = [random.random() for _ in range(20000)]
            mega_processed = [x**0.5 for x in mega_data[:5000]]
            mega_result = sum(mega_processed) / len(mega_processed)
            del mega_data, mega_processed
            
            # GPU mega-activity (if available)
            if torch.cuda.is_available():
                mega_tensor = torch.randn(500, 500).cuda()
                mega_gpu_result = mega_tensor.sum().item()
                del mega_tensor
            
            # Model mega-ping
            try:
                mega_ping = ai_service.generate_random_thought()
                print(f"🧠 MEGA model ping: {len(mega_ping)} chars generated")
            except Exception as e:
                print(f"🧠 MEGA ping warning: {e}")
            
            # Status update
            keep_alive.display_status()
        
        # Variable sleep with micro-activities
        sleep_segments = random.randint(5, 12)  # 5-12 segments
        for segment in range(sleep_segments):
            # Micro-activity during sleep
            micro_work = sum(random.randint(1, 100) for _ in range(50))
            time.sleep(random.uniform(0.5, 2.0))  # 0.5-2 second micro-sleeps
        
        # Log major milestones
        if iteration_count % 100 == 0:
            uptime = current_time - keep_alive.start_time
            print(f"🎉 MILESTONE: {iteration_count} iterations completed!")
            print(f"⏰ Total uptime: {str(uptime).split('.')[0]}")
            print(f"🔥 Total activities: {keep_alive.stats['activities_performed']}")
        
except KeyboardInterrupt:
    print("🛑 Server stopped by user")
    keep_alive.stop()
except Exception as e:
    print(f"🚨 Server error: {e}")
    print("🔄 Error logged, continuing operation...")
    time.sleep(10)  # Brief pause, then continue
